<a href="https://colab.research.google.com/github/hkit301/kaggle/blob/hajin/kakaoPrac02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
pip install surprise

In [0]:
pip install konlpy

In [0]:
%matplotlib inline
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from konlpy.tag import Twitter
from konlpy.tag import Mecab

In [0]:
genre_gn_all = pd.read_json('/content/drive/My Drive/Colab Notebooks/genre_gn_all.json', typ = 'series')
genre_gn_all = pd.DataFrame(genre_gn_all, columns = ['gnr_name']).reset_index().rename(columns = {'index' : 'gnr_code'})

In [0]:
gnr_code = genre_gn_all[genre_gn_all['gnr_code'].str[-2:] == '00']
# gnr_code

In [0]:
dtl_gnr_code = genre_gn_all[genre_gn_all['gnr_code'].str[-2:] != '00']
dtl_gnr_code.rename(columns = {'gnr_code' : 'dtl_gnr_code', 'gnr_name' : 'dtl_gnr_name'}, inplace = True)

# dtl_gnr_code

In [0]:
song_meta = pd.read_json('/content/drive/My Drive/Colab Notebooks/song_meta.json', typ = 'frame')
# song_meta

In [0]:
train = pd.read_json('/content/drive/My Drive/Colab Notebooks/train.json', typ = 'frame')

In [0]:
train = train.sort_values('like_cnt',ascending=False).reset_index()

In [0]:
train_top1000 = train.loc[0:300]

In [0]:
train_test = train_top1000
train_test

In [0]:
plylst_song_map = train_test[['id','songs']]

plylst_song_map_unset = np.dstack(
    (
        np.repeat(plylst_song_map.id.values, list(map(len, plylst_song_map.songs))), np.concatenate(plylst_song_map.songs.values)
    )
)

plylst_song_map = pd.DataFrame(data=plylst_song_map_unset[0],columns=plylst_song_map.columns)
plylst_song_map['id']=plylst_song_map['id'].astype(str)
plylst_song_map['songs']=plylst_song_map['songs'].astype(str)

del plylst_song_map_unset

In [0]:
plylst_song_cnt = pd.DataFrame(plylst_song_map.groupby('songs').id.nunique()).reset_index()

In [0]:
plylst_song_cnt.rename(columns={'songs' : 'id', 'id' : 'count'}, inplace=True)
plylst_song_cnt

In [0]:
song_meta_content = song_meta[['id','song_name','artist_name_basket']]

In [0]:
song_meta_content['id'] = song_meta_content['id'].astype(int)
plylst_song_cnt['id'] = plylst_song_cnt['id'].astype(int)
song_meta_content_cnt = song_meta_content.merge(plylst_song_cnt,on='id')

In [0]:
plylst_name = train_test[['plylst_title','songs','like_cnt','tags','id']]

In [0]:
s = plylst_name.apply(lambda x: pd.Series(x['songs']), axis=1).stack().reset_index(level=1, drop=True)
s.name = 'song'
plylst_name_meta = plylst_name.drop('songs',axis=1).join(s)

In [0]:
plylst_name_meta['song'] = plylst_name_meta['song'].astype(int)

In [0]:
s = plylst_name_meta.set_index('song')

In [0]:
s['tags'] = s['tags'].astype(str)
s['description'] = s['plylst_title'] + s['tags']

In [0]:
s = s.drop('plylst_title',axis=1)
s = s.drop('tags',axis=1)

In [0]:
s.sort_values('like_cnt',ascending=False)

In [0]:
s = s.reset_index()

In [0]:
s['song'] = s['song'].astype(int)
song_num = s[['song','description','id']]

In [0]:
song_num=song_num.rename(columns={'song' : 'song_id', 'id' : 'plylst_id'})

In [0]:
song_meta_content_cnt['id'] = song_meta_content_cnt['id'].astype(int)
song_num['song_id'] = song_num['song_id'].astype(int)

In [0]:
song_meta_content_all = pd.DataFrame.merge(song_meta_content_cnt,song_num, how='left', left_on='id', right_on='song_id')

In [0]:
song_meta_content_all.head(50)

In [0]:
twitter = Twitter()
# mecab = Mecab()

In [0]:
test = song_meta_content_all

In [0]:
test['description'] = test['description'].astype(str)
test['description'] = test['description'].apply(lambda x : twitter.nouns(x))
# test['description'] = test['description'].apply(lambda x : mecab.nouns(x))

In [0]:
test['description'] = test['description'].astype(str)

In [0]:
# tf = TfidfVectorizer(ngram_range=(1, 2), min_df=1, max_df=0.8)
# tfidf_matrix = tf.fit_transform(test['description'])

In [0]:
# tfidf_matrix.shape

In [0]:
# cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [0]:
count = CountVectorizer(ngram_range=(1,2), min_df=0)
count_matrix = count.fit_transform(test['description'])

In [0]:
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [0]:
cosine_sim[0]

In [0]:
titles = test['song_name']
indices = pd.Series(test.index, index=test['song_name'])

In [0]:
test

In [0]:
def get_recommendations(song_name) :
    idx = list(indices[song_name])
    print('메치된 곡 수 : {}\n\n'.format(len(idx)))
    # print(idx)
    for i in range(0,len(idx)):
        plylst = test.loc[idx[i]]['plylst_id']
        print('{}의 인덱스 {}번\n'.format(song_name,idx[i]))
        print('{}의 plylst ID는 {}\n'.format(song_name,plylst))
        sim_scores = list(enumerate(cosine_sim[idx[i]]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        sim_scores = sim_scores[1:31]
        song_indices = [i[0] for i in sim_scores]
        df = train_test[train_test['id']==plylst]
        songs = str(df['songs'].values)
        key = re.compile('[0-9|]+')
        songs = list(key.findall(songs))
        # songs =songs.sort()
        print(test.iloc[song_indices][['song_name','song_id']].head(10))
        print('\n')
        print(songs)
        print('\n\n')